In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
!sudo pip3 install tqdm
!sudo pip3 install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 97kB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
     |████████████████████████████████| 1.0MB 40.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=9a0154100f54ade7c1e04bf944c0b9582c185e4949fcb51eeb0ae1a897a0c0f8
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import matplotlib.pyplot as plt

%matplotlib inline

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.2.0-rc3
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/Zindi/Weekendz2/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Zindi/Weekendz2/Test.csv')
sample = pd.read_csv('/content/gdrive/My Drive/Zindi/Weekendz2/SampleSubmission.csv')

In [ ]:
train.shape, test.shape, sample.shape

((10001, 4), (5177, 2), (5177, 2))

In [ ]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [ ]:
train = train[train['label'].isnull() == False].reset_index(drop=True)
train.shape, test.shape

((10000, 4), (5177, 2))

In [ ]:
train['label'] = train['label'].round().astype(int)
train['label'].value_counts()

 0    4908
 1    4054
-1    1038
Name: label, dtype: int64

In [ ]:
test['safe_text'] = test['safe_text'].fillna('Not Available')

In [ ]:
reviews = train['safe_text'].values
sentiments = train['label'].values

train_reviews = reviews[:8000]
val_reviews = reviews [8000:]
test_reviews = test['safe_text'].values



train_sentiments = sentiments[:8000]
val_sentiments = sentiments [8000:]
# test_sentiments = sentiments[10000:]

train_reviews.shape, val_reviews.shape, test_reviews.shape

((8000,), (2000,), (5177,))

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import tqdm

def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks, all_segments= [], [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        tokens = tokenizer.tokenize(doc)
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        segments = [0] * max_seq_length
        all_ids.append(ids)
        all_masks.append(masks)
        all_segments.append(segments)
    encoded = np.array([all_ids, all_masks, all_segments])
    return encoded

In [ ]:

MAX_SEQ_LENGTH = 50

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inp_segment = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_segment_ids")
inputs = [inp_id, inp_mask, inp_segment]

hidden_state = transformers.TFBertModel.from_pretrained('bert-base-uncased')(inputs)
pooled_output = hidden_state[1]
dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(1, activation='linear')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='mean_squared_error', metrics=['mse'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
bert_segment_ids (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 50, 768), (N 109482240   bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]     

In [ ]:

train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, 
                                                                                               train_reviews, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks, val_features_segments = create_bert_input_features(tokenizer, 
                                                                                         val_reviews, 
                                                                                         max_seq_length=MAX_SEQ_LENGTH)
#test_features = create_bert_input_features(tokenizer, test_reviews, max_seq_length=MAX_SEQ_LENGTH)
print('Train Features:', train_features_ids.shape, train_features_masks.shape, train_features_segments.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape, val_features_segments.shape)

Converting docs to features: 100%|██████████| 2000/2000 [00:01<00:00, 1921.04it/s]

Train Features: (8000, 50) (8000, 50) (8000, 50)
Val Features: (2000, 50) (2000, 50) (2000, 50)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=1)
model.fit([train_features_ids, 
           train_features_masks, 
           train_features_segments], train_sentiments, 
          validation_data=([val_features_ids, 
                            val_features_masks, 
                            val_features_segments], val_sentiments),
          epochs=3, 
          batch_size=13, 
          callbacks=[es],
          shuffle=True,
          verbose=1)

Epoch 1/3
616/616 [==============================] - 89s 145ms/step - loss: 0.3878 - mse: 0.3875 - val_loss: 0.3210 - val_mse: 0.3211
Epoch 2/3
616/616 [==============================] - 86s 140ms/step - loss: 0.2923 - mse: 0.2919 - val_loss: 0.2740 - val_mse: 0.2741
Epoch 3/3
616/616 [==============================] - 86s 139ms/step - loss: 0.1889 - mse: 0.1890 - val_loss: 0.2993 - val_mse: 0.2994
Epoch 00003: early stopping


In [ ]:
model.save_weights('bert_ft_wts.h5')

In [ ]:
test_features_ids, test_features_masks, test_features_segments = create_bert_input_features(tokenizer, 
                                                                                            test_reviews, 
                                                                                            max_seq_length=MAX_SEQ_LENGTH)
print('Test Features:', test_features_ids.shape, test_features_masks.shape, test_features_segments.shape)

Converting docs to features: 100%|██████████| 5177/5177 [00:02<00:00, 1938.33it/s]

Test Features: (5177, 50) (5177, 50) (5177, 50)


In [ ]:

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predictions = model.predict([test_features_ids, 
                                            test_features_masks, 
                                            test_features_segments], verbose=0)

In [ ]:
sample['label'] = predictions
sample['label'] = np.clip(sample['label'], -1, 1)
sample['label'].describe()

count    5177.000000
mean        0.322402
std         0.375588
min        -0.942738
25%         0.042773
50%         0.250230
75%         0.641581
max         1.000000
Name: label, dtype: float64

In [ ]:
sample.to_csv('Bert_Reg.csv', index=False)

In [ ]:
sample['label'] = sample['label'].round().astype(int)
sample['label'].value_counts()

 0    3317
 1    1790
-1      70
Name: label, dtype: int64

In [ ]:
sample.to_csv('Bert_Reg_label.csv', index=False)

In [ ]:
reviews = train['safe_text'].values
sentiments = train['label'].values

train_reviews = reviews[:8000]
val_reviews = reviews [8000:]
test_reviews = test['safe_text'].values



train_sentiments = sentiments[:8000]
val_sentiments = sentiments [8000:]
# test_sentiments = sentiments[10000:]

train_reviews.shape, val_reviews.shape, test_reviews.shape

((8000,), (2000,), (5177,))

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [ ]:

import tqdm

def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks = [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        tokens = tokenizer.tokenize(doc)
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        all_ids.append(ids)
        all_masks.append(masks)
    encoded = np.array([all_ids, all_masks])
    return encoded

In [ ]:
MAX_SEQ_LENGTH = 60

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inputs = [inp_id, inp_mask]

hidden_state = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')(inputs)[0]
pooled_output = hidden_state[:, 0]    
dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(1, activation='linear')(drop2)


model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='mean_squared_error', metrics=[rmse])

model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 60)]         0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 60)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB ((None, 60, 768),)   66362880    bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]           
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 768)]        0           tf_distil_bert_model[0][0]

In [ ]:
train_features_ids, train_features_masks = create_bert_input_features(tokenizer, train_reviews, 
                                                                      max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks = create_bert_input_features(tokenizer, val_reviews, 
                                                                  max_seq_length=MAX_SEQ_LENGTH)
#test_features = create_bert_input_features(tokenizer, test_reviews, max_seq_length=MAX_SEQ_LENGTH)
print('Train Features:', train_features_ids.shape, train_features_masks.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape)

Converting docs to features: 100%|██████████| 2000/2000 [00:01<00:00, 1881.89it/s]

Train Features: (8000, 60) (8000, 60)
Val Features: (2000, 60) (2000, 60)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=2,
                                      restore_best_weights=True,
                                      verbose=1)
model.fit([train_features_ids, 
           train_features_masks], train_sentiments, 
          validation_data=([val_features_ids, 
                            val_features_masks], val_sentiments),
          epochs=10, 
          batch_size=20, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Epoch 1/10
400/400 [==============================] - 37s 92ms/step - loss: 0.3877 - rmse: 0.4832 - val_loss: 0.3340 - val_rmse: 0.4253
Epoch 2/10
400/400 [==============================] - 36s 89ms/step - loss: 0.3195 - rmse: 0.4144 - val_loss: 0.2979 - val_rmse: 0.3945
Epoch 3/10
400/400 [==============================] - 36s 90ms/step - loss: 0.2465 - rmse: 0.3538 - val_loss: 0.2908 - val_rmse: 0.3721
Epoch 4/10
400/400 [==============================] - 36s 89ms/step - loss: 0.1619 - rmse: 0.2782 - val_loss: 0.3030 - val_rmse: 0.3453
Epoch 5/10
400/400 [==============================] - 36s 89ms/step - loss: 0.1081 - rmse: 0.2243 - val_loss: 0.3125 - val_rmse: 0.3368
Epoch 00005: early stopping


In [ ]:
test_features_ids, test_features_masks = create_bert_input_features(tokenizer, test_reviews, 
                                                                    max_seq_length=MAX_SEQ_LENGTH)
print('Test Features:', test_features_ids.shape, test_features_masks.shape)

Converting docs to features: 100%|██████████| 5177/5177 [00:02<00:00, 1969.94it/s]

Test Features: (5177, 60) (5177, 60)


In [ ]:
predictions = model.predict([test_features_ids, 
                                            test_features_masks], batch_size=200, verbose=0)

In [ ]:
sample['label'] = predictions
sample['label'] = np.clip(sample['label'], -1, 1)
sample['label'].describe()

count    5177.000000
mean        0.248096
std         0.404750
min        -0.999915
25%        -0.056971
50%         0.124190
75%         0.601730
max         1.000000
Name: label, dtype: float64

In [ ]:
sample.to_csv('DistilBert_Reg.csv', index=False)

In [ ]:
from keras.utils import np_utils

In [ ]:
reviews = train['safe_text'].values
sentiments = train['label'].values

sentiments = pd.Series(sentiments).replace(-1, 2).values
sentiments = np_utils.to_categorical(sentiments)

train_reviews = reviews[:8000]
val_reviews = reviews [8000:]
test_reviews = test['safe_text'].values



train_sentiments = sentiments[:8000]
val_sentiments = sentiments [8000:]
# test_sentiments = sentiments[10000:]

train_reviews.shape, val_reviews.shape, test_reviews.shape

((8000,), (2000,), (5177,))

In [ ]:
train_sentiments

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
MAX_SEQ_LENGTH = 100

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inp_segment = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_segment_ids")
inputs = [inp_id, inp_mask, inp_segment]

hidden_state = transformers.TFBertModel.from_pretrained('bert-base-uncased')(inputs)
pooled_output = hidden_state[1]
dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(3, activation='softmax')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert_segment_ids (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   ((None, 100, 768), ( 109482240   bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]     

In [ ]:
train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, 
                                                                                               train_reviews, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks, val_features_segments = create_bert_input_features(tokenizer, 
                                                                                         val_reviews, 
                                                                                         max_seq_length=MAX_SEQ_LENGTH)
#test_features = create_bert_input_features(tokenizer, test_reviews, max_seq_length=MAX_SEQ_LENGTH)
print('Train Features:', train_features_ids.shape, train_features_masks.shape, train_features_segments.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape, val_features_segments.shape)

Converting docs to features: 100%|██████████| 2000/2000 [00:01<00:00, 1889.64it/s]


Train Features: (8000, 100) (8000, 100) (8000, 100)
Val Features: (2000, 100) (2000, 100) (2000, 100)


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=1)
model.fit([train_features_ids, 
           train_features_masks, 
           train_features_segments], train_sentiments, 
          validation_data=([val_features_ids, 
                            val_features_masks, 
                            val_features_segments], val_sentiments),
          epochs=3, 
          batch_size=13, 
          callbacks=[es],
          shuffle=True,
          verbose=1)

Epoch 1/3
616/616 [==============================] - 133s 215ms/step - loss: 0.7320 - accuracy: 0.6874 - val_loss: 0.6427 - val_accuracy: 0.7180
Epoch 2/3
616/616 [==============================] - 130s 210ms/step - loss: 0.5430 - accuracy: 0.7818 - val_loss: 0.5702 - val_accuracy: 0.7845
Epoch 3/3
616/616 [==============================] - 130s 211ms/step - loss: 0.3818 - accuracy: 0.8608 - val_loss: 0.5802 - val_accuracy: 0.7795
Epoch 00003: early stopping


In [ ]:

test_features_ids, test_features_masks, test_features_segments = create_bert_input_features(tokenizer, 
                                                                                            test_reviews, 
                                                                                            max_seq_length=MAX_SEQ_LENGTH)
print('Test Features:', test_features_ids.shape, test_features_masks.shape, test_features_segments.shape)

Converting docs to features: 100%|██████████| 5177/5177 [00:02<00:00, 1924.17it/s]

Test Features: (5177, 100) (5177, 100) (5177, 100)


In [ ]:
predictions = model.predict([test_features_ids, 
                                            test_features_masks, 
                                            test_features_segments], verbose=0)

In [ ]:
sample['label'] = predictions.argmax(axis=1)
sample['label'] = sample['label'].replace(2, -1)
sample['label'].value_counts()

 1    2383
 0    2232
-1     562
Name: label, dtype: int64

In [ ]:
sample.to_csv("Bert_1.csv", index=False)